In [1]:
!pip install git+https://github.com/shahashka/SP-GIES

  Cloning https://github.com/shahashka/SP-GIES to /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-c2l433ax
  Running command git clone --filter=blob:none --quiet https://github.com/shahashka/SP-GIES /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-c2l433ax
  Resolved https://github.com/shahashka/SP-GIES to commit d6e9183cfba5d4986f68be811fd26e4ed8ae4df7
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... - done


In [2]:
from sp_gies import utils
from sp_gies import sp_gies as sp

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
# Generate a random graph and random data.
# Data is sampled from a linear Gaussian model (child nodes are weighted sums of parent nodes)
# Arcs is a list of edge pairs e.g. [("G1","G2"), ("G2","G4")]
(arcs,nodes,bias,var), data = utils.get_random_graph_data("scale_free", n=10, nsamples=10000, iv_samples=10, p=0.5, k=2)

In [4]:
# The dataset should have column names corresponding to the random variables names
print(data.head)

<bound method NDFrame.head of            G1        G2        G3        G4        G5        G6        G7   
0    4.979324  1.880025 -0.972235 -0.035766 -0.171051  3.007217  3.753972  \
1   -1.947711 -1.385155 -1.777715 -0.863032 -1.689290 -0.195150  1.476377   
2    3.285784 -0.669466  0.748082 -0.928084  0.495895  2.912336  1.540072   
3   16.498070  5.259697  2.345555 -0.349151  3.796432  7.156625  8.851658   
4   12.602891  4.641528  1.185216  0.410582  0.554082  3.680665  8.411329   
..        ...       ...       ...       ...       ...       ...       ...   
5    2.746896  0.814001  1.452603  0.341534 -0.286683  1.093233  0.432757   
6    4.078718  0.425707  1.296373 -0.718520  0.959463  2.762145  3.060405   
7   -2.903208 -0.137007 -2.210221 -0.810198 -0.815238 -0.079134 -0.191645   
8   10.946452  6.191972  1.565690 -0.275620  2.392486  4.202782  7.442484   
9    0.096591 -1.977975  0.752584 -0.156714 -0.496039  2.156474 -0.960126   

          G8        G9       G10  target  
0 

In [9]:
# Run the SP-GIES algorithm on the dataset.
# Adjacency matrix is saved in the output dir
# Optionally provide a skeleton, which should be a numpy array with number rows/columns corresponding to the number of nodes
# For a 10 node graph this should take less than 1 second
adj_mat = sp.sp_gies(data, outdir="./", skel=None, pc=True)

TypeError: sp_gies() got an unexpected keyword argument 'pc'

In [10]:
# Check the accuracy of the estimated graph
# Prints the SHD, SID, and AUC of the estimated graph against the true generated graph
G_true = utils.edge_to_dag(arcs)
G_sp_gies = utils.adj_to_dag(adj_mat, nodes)
print(utils.get_scores(["SP-GIES"], [G_sp_gies], G_true, get_sid=True))

NameError: name 'adj_mat' is not defined